In [ ]:
import requests
from bs4 import BeautifulSoup #backing is web scraping 
import time 
import pandas as pd
import csv
import json

In [ ]:

url = "https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/"  #لينك website بتاعتي 


HEADERS = {                                                          #HEADERS: بيانات إضافية بتتبعت مع الطلب علشان الموقع يعرف مين بيطلب الصفحة
    "User-Agent": (                                                  #User-Agent: جزء من الـ headers بيقول للموقع نوع المتصفح والجهاز اللي بيطلب 
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "                 #ليه بنستخدمUser-Agent؟
        "AppleWebKit/537.36 (KHTML, like Gecko) "                    #علشان بعض المواقع بتحظر طلبات  Python
        "Chrome/119.0.0.0 Safari/537.36"                             #لما نحط User-Agent حقيقي، الموقع يفتكر إن الطلب جاي من متصفح عادي → فمش هيعمل block أو يظهر خطأ
    )
}

In [ ]:
def fetch_html(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=15)       #بتبعت طلب GET للموقع
                                                                    #بتحط الـ headers
                                                                    #بنحط time لو الطلب ف خلال الفتره ده متقبلشي cancell

        if resp.status_code == 200:                                 # parser :أداة داخل بايثون بتقرأ صفحة html  ويحولها الي شكل منظم اقدر اتعامل معاها 
            return BeautifulSoup(resp.text, "html.parser")          #BeautifulSoup: ليه بنستخدمه؟
                                                                    #يعرف يحدد العناصر (div, a, ul...)\\\\ يسمحلك تبحث جوّاها بسهولة بـ .find() و .find_all()
      
        else:            
            print(f"[warn] {url} - HTTP {resp.status_code}")        #بيطبع تحذير مع رقم الخطأ (زي 404 أو 500)
            return None                                             #بيرجع None
        
    except requests.RequestException as e:                          #لو حصل أي خطأ أثناء الاتصال بالموقع (انقطاع نت)
        print(f"[error] Request failed for {url}: {e}")             #بيطبع Error message
        return None                                    

In [ ]:
soup = fetch_html(url)                                              #استدعاء المكتبه 
soup

<!DOCTYPE html>

<html lang="en-GB">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=2" name="viewport"/>
<link href="https://appnhs24wp41a8c38064.blob.core.windows.net/blobappnhs24wp41a8c38064/wp-content/themes/nhsinform.bk.27_11_2025/assets/css/global.css" rel="stylesheet"/>
<!-- <meta http-equiv="Content-Security-Policy"  content="script-src 'unsafe-inline'; style-src 'unsafe-inline'"> -->
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
<style>img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }</style>
<!-- This site is optimized with the Yoast SEO plugin v26.4 - https://yoast.com/wordpress/plugins/seo/ -->
<title>A to Z list of common illnesses and conditions | NHS inform</title>
<meta content="A to Z list of

##################################################################################################################################

In [ ]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class":"az_list_indivisual a_detail"})  #دورلي على كل <div> في الصفحة اللي عنده الكلاس ده الا عندي  
    
    for i  in class_list:
        letter = i.find("h2").text.strip()                                      #جوّه كل div… في <h2> بيكون مكتوب فيه الحرف
        ul = i.find("ul")                                                       #كل حرف تحته قائمة <ul>
                                                                                #جواها <li> كل واحد فيه مرض 

        if ul:
            for li in ul.find_all("li"):                                        #المرور على كل مرض 
                a = li.find("a")
                if a:
                    name = a.text.strip()                                       #طلعلي الاسم 
                    link = a.get("href")                                        #طلعلي الرابط 
                    rows.append((letter, name, link))                           #وضيفيهم هنا 
    return rows                                                                 #إرجاع النتايج كلها



disease_data = parse_page(soup)                             
for item in disease_data[:]:                                                    #هنا بقوله عدي علي كل مرض 
    print(item)

('A', 'Abdominal aortic aneurysm', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/abdominal-aortic-aneurysm/')
('A', 'Achilles tendinopathy', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/achilles-tendinopathy/')
('A', 'Acne', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/acne/')
('A', 'Acute cholecystitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/acute-cholecystitis/')
('A', 'Acute lymphoblastic leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/acute-lymphoblastic-leukaemia/')
('A', 'Acute lymphoblastic leukaemia: Children', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/acute-lymphoblastic-leukaemia-children/')
('A', 'Acute myeloid leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/acute-myeloid-leuka

In [ ]:

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)                              # إرسال طلب للموقع مع الهيدر
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):                                         # بنمشي على كل H2 
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None         # تحديد الـ H2 اللي بعده علشان نعرف أين ينتهي هذا القسم

        paragraphs = []                                                        # هنخزن فقرات هذا القسم هنا
        for sibling in h2.find_next_siblings():                                #         نجيب كل العناصر اللي بعد الـ H2 لحد ما نوصل للـ H2 اللي بعده

            if sibling == next_h2:                                             # لو وصلنا لعنوان جديد نوقف 
                break
            if sibling.name == "p": 
                text = sibling.get_text(strip=True)
                if text and text != ".":                                       #                 نتأكد أن النص مش فاضي

                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs                                           #فقط لو القسم يحتوي على فقرات نخزّنه 

    return data

# قائمة الروابط
disease_links = [
    ('A', 'Abdominal aortic aneurysm', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/abdominal-aortic-aneurysm/'),
    ('A', 'Achilles tendinopathy', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/achilles-tendinopathy/'),
    ('A', 'Acne', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/acne/'),
    ('A', 'Acute cholecystitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/acute-cholecystitis/'),
    ('A', 'Acute lymphoblastic leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/acute-lymphoblastic-leukaemia/'),
    ('A', 'Acute lymphoblastic leukaemia: Children', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/acute-lymphoblastic-leukaemia-children/'),
    ('A', 'Acute myeloid leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/acute-myeloid-leukaemia/'),
    ('A', 'Acute myeloid leukaemia: Children', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/acute-myeloid-leukaemia-children/'),
    ('A', 'Acute pancreatitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/acute-pancreatitis/'),
    ('A', 'Acute respiratory infection (ARI)', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/acute-respiratory-infection-ari/'),
    ('A', 'Addison’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/glands/addisons-disease/'),
    ('A', 'Adenomyosis', 'https://www.nhsinform.scot/healthy-living/womens-health/girls-and-young-women-puberty-to-around-25/periods-and-menstrual-health/adenomyosis/'),
    ('A', 'Alcohol-related liver disease', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/alcohol-related-liver-disease/'),
    ('A', 'Allergic rhinitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/allergic-rhinitis/'),
    ('A', 'Allergies', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/allergies/'),
    ('A', 'Alzheimer’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia/types-of-dementia/alzheimers-disease/'),
    ('A', 'Anal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/anal-cancer/'),
    ('A', 'Anaphylaxis', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/anaphylaxis/'),
    ('A', 'Angina', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/angina/'),
    ('A', 'Angioedema', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/angioedema/'),
    ('A', 'Ankle sprain', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/ankle-sprain/'),
    ('A', 'Ankle avulsion fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/ankle-avulsion-fracture/'),
    ('A', 'Ankylosing spondylitis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/ankylosing-spondylitis/'),
    ('A', 'Anorexia nervosa', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/anorexia-nervosa/'),
    ('A', 'Anxiety disorders in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/anxiety-disorders-in-children-and-young-people/'),
    ('A', 'Aplastic anaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/aplastic-anaemia/'),
    ('A', 'Appendicitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/appendicitis/'),
    ('A', 'Arterial thrombosis', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/arterial-thrombosis/'),
    ('A', 'Arthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/arthritis/'),
    ('A', 'Asbestosis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/asbestosis/'),
    ('A', 'Asthma', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/asthma/'),
    ('A', 'Ataxia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/ataxia/'),
    ('A', 'Atopic eczema', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/atopic-eczema/'),
    ('A', 'Atrial fibrillation', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/atrial-fibrillation/'),
    ('A', 'Attention deficit hyperactivity disorder (ADHD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/attention-deficit-hyperactivity-disorder-adhd/'),
    ('A', 'Autism', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/autism')
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_A = {}                                                 #هنا هنخزن بيانات الأمراض كلها 

for letter, name, link in disease_links:                                 ## نكرر على كل مرض في القائمة

    print(f"Processing {name}...")                                       #رساله الpreprocssing
    sections = get_sections_key_value(link)                              #     ننادي الدالة علشان تجيب أقسام المرض وفقراته

    all_diseases_data_A[name] = sections                                 #     نحفظ النتيجة باسم المرض


# حفظ النتيجة في ملف JSON
with open("all_diseases_data_A.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_A, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Abdominal aortic aneurysm...
Processing Achilles tendinopathy...
Processing Acne...
Processing Acute cholecystitis...
Processing Acute lymphoblastic leukaemia...
Processing Acute lymphoblastic leukaemia: Children...
Processing Acute myeloid leukaemia...
Processing Acute myeloid leukaemia: Children...
Processing Acute pancreatitis...
Processing Acute respiratory infection (ARI)...
Processing Addison’s disease...
Processing Adenomyosis...
Processing Alcohol-related liver disease...
Processing Allergic rhinitis...
Processing Allergies...
Processing Alzheimer’s disease...
Processing Anal cancer...
Processing Anaphylaxis...
Processing Angina...
Processing Angioedema...
Processing Ankle sprain...
Processing Ankle avulsion fracture...
Processing Ankylosing spondylitis...
Processing Anorexia nervosa...
Processing Anxiety disorders in children and young people...
Processing Aplastic anaemia...
Processing Appendicitis...
Processing Arterial thrombosis...
Processing Arthritis...
Proces

#######################################################################################################################################################

In [7]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual b_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('B', 'Back problems', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/back-problems/')
('B', 'Bacterial vaginosis', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/bacterial-vaginosis/')
('B', 'Becker muscular dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/becker-muscular-dystrophy/')
('B', 'Benign prostate enlargement', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/benign-prostate-enlargement/')
('B', 'Bile duct cancer (cholangiocarcinoma)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/bile-duct-cancer-cholangiocarcinoma/')
('B', 'Binge eating disorder (BED)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/binge-eating-disorder-bed/')
('B', 'Bipolar disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/ment

In [8]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('B', 'Back problems', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/back-problems/'),
('B', 'Bacterial vaginosis', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/bacterial-vaginosis/'),
('B', 'Becker muscular dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/becker-muscular-dystrophy/'),
('B', 'Benign prostate enlargement', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/benign-prostate-enlargement/'),
('B', 'Bile duct cancer (cholangiocarcinoma)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/bile-duct-cancer-cholangiocarcinoma/'),
('B', 'Binge eating disorder (BED)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/binge-eating-disorder-bed/'),
('B', 'Bipolar disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/bipolar-disorder/'),
('B', 'Bladder cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/bladder-cancer/'),
('B', 'Blood poisoning (sepsis)', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/sepsis/'),
('B', 'Bone cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/bone-cancer/'),
('B', 'Bowel cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/bowel-cancer/'),
('B', 'Bowel incontinence', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/bowel-incontinence/'),
('B', 'Bowel polyps', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/bowel-polyps/'),
('B', 'Bow legs and knock knees in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscle-bone-and-joints/bow-legs-and-knock-knees-in-children-and-young-people/'),
('B', 'Brain stem death', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/brain-stem-death/'),
('B', 'Brain tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/brain-tumours/'),
('B', 'Brain tumours: Children', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/brain-tumours-children/'),
('B', 'Brain tumours: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/brain-tumours-teenagers-and-young-adults/'),
('B', 'Breast cancer (female)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/breast-cancer-female/'),
('B', 'Breast cancer (male)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/breast-cancer-male/'),
('B', 'Breathing problems in children', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/breathing-problems-in-children/'),
('B', 'Breathlessness', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/shortness-of-breath/'),
('B', 'Bronchiectasis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/bronchiectasis/'),
('B', 'Bronchitis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/bronchitis/'),
('B', 'Bulimia nervosa', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/bulimia-nervosa/'),
('B', 'Bunion (hallux valgus)', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/bunion-hallux-valgus/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_B = {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_B[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_B.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_B, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Back problems...
Processing Bacterial vaginosis...
Processing Becker muscular dystrophy...
Processing Benign prostate enlargement...
Processing Bile duct cancer (cholangiocarcinoma)...
Processing Binge eating disorder (BED)...
Processing Bipolar disorder...
Processing Bladder cancer...
Processing Blood poisoning (sepsis)...
Processing Bone cancer...
Processing Bowel cancer...
Processing Bowel incontinence...
Processing Bowel polyps...
Processing Bow legs and knock knees in children and young people...
Processing Brain stem death...
Processing Brain tumours...
Processing Brain tumours: Children...
Processing Brain tumours: Teenagers and young adults...
Processing Breast cancer (female)...
Processing Breast cancer (male)...
Processing Breathing problems in children...
Processing Breathlessness...
Processing Bronchiectasis...
Processing Bronchitis...
Processing Bulimia nervosa...
Processing Bunion (hallux valgus)...
All data saved to all_diseases_data.json


#####################################################################################################################################

In [9]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual c_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('C', 'Carcinoid syndrome and carcinoid tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/carcinoid-syndrome-and-carcinoid-tumours/')
('C', 'Cardiac arrest', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/cardiac-arrest/')
('C', 'Cardiovascular disease', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/')
('C', 'Carpal tunnel syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/carpal-tunnel-syndrome/')
('C', 'Catarrh', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/catarrh/')
('C', 'Cellulitis', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/cellulitis/')
('C', 'Cerebral palsy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/cerebral-palsy/')
('C', 'Cervical cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer

In [10]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('C', 'Carcinoid syndrome and carcinoid tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/carcinoid-syndrome-and-carcinoid-tumours/'),
('C', 'Cardiac arrest', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/cardiac-arrest/'),
('C', 'Cardiovascular disease', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/'),
('C', 'Carpal tunnel syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/carpal-tunnel-syndrome/'),
('C', 'Catarrh', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/catarrh/'),
('C', 'Cellulitis', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/cellulitis/'),
('C', 'Cerebral palsy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/cerebral-palsy/'),
('C', 'Cervical cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/cervical-cancer/'),
('C', 'Cervical spondylosis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/cervical-spondylosis/'),
('C', 'Chest and rib injury', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/chest-and-rib-problems-and-conditions/chest-and-rib-injury/'),
('C', 'Chest infection', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/chest-infection/'),
('C', 'Chickenpox', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/chickenpox/'),
('C', 'Chilblains', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/chilblains/'),
('C', 'Chlamydia', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/chlamydia/'),
('C', 'Chronic fatigue syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/myalgic-encephalomyelitis-me-or-chronic-fatigue-syndrome-cfs/'),
('C', 'Chronic kidney disease', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/chronic-kidney-disease/'),
('C', 'Chronic lymphocytic leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/chronic-lymphocytic-leukaemia/'),
('C', 'Chronic myeloid leukaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/chronic-myeloid-leukaemia/'),
('C', 'Chronic obstructive pulmonary disease (COPD)', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/copd/chronic-obstructive-pulmonary-disease/'),
('C', 'Chronic pain', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/chronic-pain'),
('C', 'Chronic pancreatitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/chronic-pancreatitis/'),
('C', 'Cirrhosis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/cirrhosis/'),
('C', 'Clavicle (collar bone) fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/chest-and-rib-problems-and-conditions/clavicle-collar-bone-fracture/'),
('C', 'Clostridium difficile', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/clostridium-difficile/'),
('C', 'Coeliac disease', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/coeliac-disease/coeliac-disease/'),
('C', 'Cold sore', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/cold-sore/'),
('C', 'Coma', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/coma/'),
('C', 'Common cold', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/common-cold/'),
('C', 'Complications of type 1 diabetes', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/complications-of-type-1-diabetes/'),
('C', 'Concussion', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/concussion/'),
('C', 'Congenital heart disease', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/congenital-heart-disease/'),
('C', 'Congenital muscular dystrophy (CMD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/congenital-muscular-dystrophy-cmd/'),
('C', 'Conjunctivitis', 'https://www.nhsinform.scot/illnesses-and-conditions/eyes/conjunctivitis/'),
('C', 'Constipation', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/constipation/'),
('C', 'Coronary heart disease', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/coronary-heart-disease/'),
('C', 'Coronavirus (COVID-19)', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/coronavirus-covid-19/coronavirus-covid-19/'),
('C', 'Coronavirus (COVID-19): Longer-term effects (long COVID)', 'https://www.nhsinform.scot/long-term-effects-of-covid-19-long-covid/'),
('C', 'Costochondritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/chest-and-rib-problems-and-conditions/costochondritis/'),
('C', 'Cough', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/cough/'),
('C', 'Crohn’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/inflammatory-bowel-disease-ibd/crohns-disease/'),
('C', 'Croup', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/croup/'),
('C', 'Cystic fibrosis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/cystic-fibrosis/'),
('C', 'Cystitis', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/cystitis/'),

]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_C = {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_C[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_C.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_C, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Carcinoid syndrome and carcinoid tumours...
Processing Cardiac arrest...
Processing Cardiovascular disease...
Processing Carpal tunnel syndrome...
Processing Catarrh...
Processing Cellulitis...
Processing Cerebral palsy...
Processing Cervical cancer...
Processing Cervical spondylosis...
Processing Chest and rib injury...
Processing Chest infection...
Processing Chickenpox...
Processing Chilblains...
Processing Chlamydia...
Processing Chronic fatigue syndrome...
Processing Chronic kidney disease...
Processing Chronic lymphocytic leukaemia...
Processing Chronic myeloid leukaemia...
Processing Chronic obstructive pulmonary disease (COPD)...
Processing Chronic pain...
Processing Chronic pancreatitis...
Processing Cirrhosis...
Processing Clavicle (collar bone) fracture...
Processing Clostridium difficile...
Processing Coeliac disease...
Processing Cold sore...
Processing Coma...
Processing Common cold...
Processing Complications of type 1 diabetes...
Processing Concussion...
Proc

##########################################################################################################################

In [11]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual d_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('D', 'Deafblindness', 'https://www.nhsinform.scot/illnesses-and-conditions/eyes/deafblindness/')
('D', 'Deep vein thrombosis', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/deep-vein-thrombosis/')
('D', 'Degenerative cervical myelopathy', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/degenerative-cervical-myelopathy/')
('D', 'Dehydration', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/dehydration/')
('D', 'Delirium', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/delirium/')
('D', 'Dementia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia')
('D', 'Dementia with Lewy bodies', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia/types-of-dementia/dementia-with-lewy-bodies/')
('D', 'Dental abscess', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/dental-

In [12]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('D', 'Deafblindness', 'https://www.nhsinform.scot/illnesses-and-conditions/eyes/deafblindness/'),
('D', 'Deep vein thrombosis', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/deep-vein-thrombosis/'),
('D', 'Degenerative cervical myelopathy', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/degenerative-cervical-myelopathy/'),
('D', 'Dehydration', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/dehydration/'),
('D', 'Delirium', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/delirium/'),
('D', 'Dementia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia'),
('D', 'Dementia with Lewy bodies', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia/types-of-dementia/dementia-with-lewy-bodies/'),
('D', 'Dental abscess', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/dental-abscess/'),
('D', 'Depression', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/depression/'),
('D', 'Dermatitis herpetiformis', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/dermatitis-herpetiformis/'),
('D', 'Diabetic foot issues', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/diabetic-foot-issues/'),
('D', 'Diabetic ketoacidosis (DKA)', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/diabetic-ketoacidosis-dka/'),
('D', 'Diabetic retinopathy', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/diabetic-retinopathy/'),
('D', 'Diarrhoea in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/diarrhoea-in-adults/'),
('D', 'Diarrhoea in children and babies', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/diarrhoea-in-children-and-babies/'),
('D', 'Discoid eczema', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/discoid-eczema/'),
('D', 'Diverticular disease and diverticulitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/diverticular-disease-and-diverticulitis/'),
('D', 'Dizziness (Lightheadedness)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dizziness-lightheadedness/'),
('D', 'Down’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/downs-syndrome/'),
('D', 'Dry mouth', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/dry-mouth/'),
('D', 'Duchenne muscular dystrophy (DMD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/duchenne-muscular-dystrophy-dmd/'),
('D', 'Dysphagia (swallowing problems)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/dysphagia-swallowing-problems/'),
('D', 'Dystonia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dystonia/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_D= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_D[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_D.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_D, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Deafblindness...
Processing Deep vein thrombosis...
Processing Degenerative cervical myelopathy...
Processing Dehydration...
Processing Delirium...
Processing Dementia...
Processing Dementia with Lewy bodies...
Processing Dental abscess...
Processing Depression...
Processing Dermatitis herpetiformis...
Processing Diabetic foot issues...
Processing Diabetic ketoacidosis (DKA)...
Processing Diabetic retinopathy...
Processing Diarrhoea in adults...
Processing Diarrhoea in children and babies...
Processing Discoid eczema...
Processing Diverticular disease and diverticulitis...
Processing Dizziness (Lightheadedness)...
Processing Down’s syndrome...
Processing Dry mouth...
Processing Duchenne muscular dystrophy (DMD)...
Processing Dysphagia (swallowing problems)...
Processing Dystonia...
All data saved to all_diseases_data.json


#################################################################################################################################################

In [13]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual e_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('E', 'Eating disorders', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/')
('E', 'Earache', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/earache/')
('E', 'Early miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/early-miscarriage/')
('E', 'Earwax build-up', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/earwax-build-up/')
('E', 'Ebola virus disease', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/ebola-virus-disease/')
('E', 'Ectopic pregnancy', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/ectopic-pregnancy/')
('E', 'Elbow (radial head or neck) fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/elbow-radial-head-or-neck-fracture/')
('E', 'Edwards’ syndrome', 'https://www.nhsinform.s

In [14]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('E', 'Eating disorders', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/eating-disorders/'),
('E', 'Earache', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/earache/'),
('E', 'Early miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/early-miscarriage/'),
('E', 'Earwax build-up', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/earwax-build-up/'),
('E', 'Ebola virus disease', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/ebola-virus-disease/'),
('E', 'Ectopic pregnancy', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/ectopic-pregnancy/'),
('E', 'Elbow (radial head or neck) fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/elbow-radial-head-or-neck-fracture/'),
('E', 'Edwards’ syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/chromosomal-conditions/edwards-syndrome/'),
('E', 'Emery-Dreifuss muscular dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/emery-dreifuss-muscular-dystrophy/'),
('E', 'Endometriosis', 'https://www.nhsinform.scot/healthy-living/womens-health/girls-and-young-women-puberty-to-around-25/periods-and-menstrual-health/endometriosis/'),
('E', 'Epilepsy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/epilepsy/'),
('E', 'Erectile dysfunction (impotence)', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/erectile-dysfunction-impotence/'),
('E', 'Escherichia coli (E. coli) O157', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/escherichia-coli-e-coli-o157/'),
('E', 'Ewing sarcoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/ewing-sarcoma/'),
('E', 'Ewing sarcoma: Children', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/ewing-sarcoma-children/'),
('E', 'Eye cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/eye-cancer/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_E= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_E[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_E.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_E, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Eating disorders...
Processing Earache...
Processing Early miscarriage...
Processing Earwax build-up...
Processing Ebola virus disease...
Processing Ectopic pregnancy...
Processing Elbow (radial head or neck) fracture...
Processing Edwards’ syndrome...
Processing Emery-Dreifuss muscular dystrophy...
Processing Endometriosis...
Processing Epilepsy...
Processing Erectile dysfunction (impotence)...
Processing Escherichia coli (E. coli) O157...
Processing Ewing sarcoma...
Processing Ewing sarcoma: Children...
Processing Eye cancer...
All data saved to all_diseases_data.json


#######################################################################################################################################

In [15]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual f_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('F', 'Facioscapulohumeral muscular dystrophy (FSHD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/facioscapulohumeral-muscular-dystrophy-fshd/')
('F', 'Farting', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/farting/')
('F', 'Febrile seizures', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/febrile-seizures/')
('F', 'Feeling of something in your throat (Globus)', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/feeling-of-something-in-your-throat-globus/')
('F', 'Fever in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/fever-in-adults/')
('F', 'Fever in children', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/fever-in-children/')
('F', 'Fibroids', 'https://www.nhsinform.scot/healthy-living/womens-health/girls-and-young-women-puberty-to-around-25/periods

In [16]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('F', 'Facioscapulohumeral muscular dystrophy (FSHD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/facioscapulohumeral-muscular-dystrophy-fshd/'),
('F', 'Farting', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/farting/'),
('F', 'Febrile seizures', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/febrile-seizures/'),
('F', 'Feeling of something in your throat (Globus)', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/feeling-of-something-in-your-throat-globus/'),
('F', 'Fever in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/fever-in-adults/'),
('F', 'Fever in children', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/fever-in-children/'),
('F', 'Fibroids', 'https://www.nhsinform.scot/healthy-living/womens-health/girls-and-young-women-puberty-to-around-25/periods-and-menstrual-health/fibroids/'),
('F', 'Fibromyalgia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/chronic-pain/fibromyalgia/'),
('F', 'Flat feet in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/flat-feet-in-children-and-young-people/'),
('F', 'Flu', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/flu/'),
('F', 'Foetal alcohol syndrome', 'https://www.nhsinform.scot/ready-steady-baby/pregnancy/looking-after-yourself-and-your-baby/alcohol-and-pregnancy/'),
('F', 'Food allergy', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/food-allergy/'),
('F', 'Food poisoning', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/food-poisoning/'),
('F', 'Fragility fracture of the hip', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/fragility-fracture-of-the-hip/'),
('F', 'Frozen shoulder', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/frozen-shoulder/'),
('F', 'Functional neurological disorder (FND)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/functional-neurological-disorder/'),
('F', 'Fungal nail infection', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/fungal-nail-infection/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_F= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_F[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_F.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_F, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Facioscapulohumeral muscular dystrophy (FSHD)...
Processing Farting...
Processing Febrile seizures...
Processing Feeling of something in your throat (Globus)...
Processing Fever in adults...
Processing Fever in children...
Processing Fibroids...
Processing Fibromyalgia...
Processing Flat feet in children and young people...
Processing Flu...
Processing Foetal alcohol syndrome...
Processing Food allergy...
Processing Food poisoning...
Processing Fragility fracture of the hip...
Processing Frozen shoulder...
Processing Functional neurological disorder (FND)...
Processing Fungal nail infection...
All data saved to all_diseases_data.json


####################################################################################################################################

In [17]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual g_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('G', 'Gallbladder cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/gallbladder-cancer/')
('G', 'Gallstones', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gallstones/')
('G', 'Ganglion cyst', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/ganglion-cyst/')
('G', 'Ganglion cysts in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/ganglion-cysts-in-children-and-young-people/')
('G', 'Gastroenteritis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gastroenteritis/')
('G', 'Gastro-oesophageal reflux disease (GORD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gastro-oesophageal-reflux-disease-gord/')
('G', 'Generalised anxiet

In [19]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('G', 'Gallbladder cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/gallbladder-cancer/'),
('G', 'Gallstones', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gallstones/'),
('G', 'Ganglion cyst', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/ganglion-cyst/'),
('G', 'Ganglion cysts in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/ganglion-cysts-in-children-and-young-people/'),
('G', 'Gastroenteritis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gastroenteritis/'),
('G', 'Gastro-oesophageal reflux disease (GORD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/gastro-oesophageal-reflux-disease-gord/'),
('G', 'Generalised anxiety disorder (GAD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/generalised-anxiety-disorder-gad/'),
('G', 'Genital herpes', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/genital-herpes/'),
('G', 'Genital symptoms', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/managing-genital-symptoms/'),
('G', 'Genital warts', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/genital-warts/'),
('G', 'Germ cell tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/germ-cell-tumours/'),
('G', 'Glandular fever', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/glandular-fever/'),
('G', 'Golfers elbow', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/golfers-elbow/'),
('G', 'Gonorrhoea', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/gonorrhoea/'),
('G', 'Gout', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/gout/'),
('G', 'Greater trochanteric pain syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/greater-trochanteric-pain-syndrome/'),
('G', 'Gum disease', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/gum-disease/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_G= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_G[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_G.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_G, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Gallbladder cancer...
Processing Gallstones...
Processing Ganglion cyst...
Processing Ganglion cysts in children and young people...
Processing Gastroenteritis...
Processing Gastro-oesophageal reflux disease (GORD)...
Processing Generalised anxiety disorder (GAD)...
Processing Genital herpes...
Processing Genital symptoms...
Processing Genital warts...
Processing Germ cell tumours...
Processing Glandular fever...
Processing Golfers elbow...
Processing Gonorrhoea...
Processing Gout...
Processing Greater trochanteric pain syndrome...
Processing Gum disease...
All data saved to all_diseases_data.json


##########################################################################################################################################################################################3

In [21]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual h_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('H', 'Haemorrhoids (piles)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/haemorrhoids-piles/')
('H', 'Hand, foot and mouth disease', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/hand-foot-and-mouth-disease/')
('H', 'Hay fever', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/hay-fever/')
('H', 'Head and neck cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/head-and-neck-cancer/')
('H', 'Head lice and nits', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/head-lice-and-nits/')
('H', 'Headaches', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/headaches/')
('H', 'Hearing loss', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/hearing-loss/')
('H', 'Heart attack', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/heart-atta

In [22]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('H', 'Haemorrhoids (piles)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/haemorrhoids-piles/'),
('H', 'Hand, foot and mouth disease', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/hand-foot-and-mouth-disease/'),
('H', 'Hay fever', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/hay-fever/'),
('H', 'Head and neck cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/head-and-neck-cancer/'),
('H', 'Head lice and nits', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/head-lice-and-nits/'),
('H', 'Headaches', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/headaches/'),
('H', 'Hearing loss', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/hearing-loss/'),
('H', 'Heart attack', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/heart-attack/'),
('H', 'Heart block', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/heart-block/'),
('H', 'Heart disease', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/'),
('H', 'Heart failure', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/heart-failure/'),
('H', 'Heart palpitations', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/heart-palpitations/'),
('H', 'Heatstroke and heat illness', 'https://www.nhsinform.scot/illnesses-and-conditions/a-z/heatstroke-and-heat-illness/'),
('H', 'Hepatitis A', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/hepatitis-a/'),
('H', 'Hepatitis B', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/hepatitis-b/'),
('H', 'Hepatitis C', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/hepatitis-c/'),
('H', 'Hiatus hernia', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/hiatus-hernia/'),
('H', 'High blood pressure (hypertension)', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/risk-factors-for-cardiovascular-disease/high-blood-pressure-hypertension/'),
('H', 'High cholesterol', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/risk-factors-for-cardiovascular-disease/high-cholesterol/'),
('H', 'HIV', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/hiv/'),
('H', 'Hives', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/hives/'),
('H', 'Hodgkin lymphoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/hodgkin-lymphoma/'),
('H', 'Hodgkin lymphoma: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/hodgkin-lymphoma-teenagers-and-young-adults/'),
('H', 'Huntington’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/huntingtons-disease/'),
('H', 'Hydrocephalus', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/hydrocephalus/'),
('H', 'Hyperglycaemia (high blood sugar)', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/hyperglycaemia-high-blood-sugar/'),
('H', 'Hyperhidrosis', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/hyperhidrosis/'),
('H', 'Hypoglycaemia (low blood sugar)', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/hypoglycaemia-low-blood-sugar/'),
('H', 'Hypothermia (low body temperature)', 'https://www.nhsinform.scot/illnesses-and-conditions/a-z/hypothermia-low-body-temperature/'),
]

# إنشاء JSON واحد لكل الأمراض
all_diseases_data_H= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_H[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_H.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_H, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Haemorrhoids (piles)...
Processing Hand, foot and mouth disease...
Processing Hay fever...
Processing Head and neck cancer...
Processing Head lice and nits...
Processing Headaches...
Processing Hearing loss...
Processing Heart attack...
Processing Heart block...
Processing Heart disease...
Processing Heart failure...
Processing Heart palpitations...
Processing Heatstroke and heat illness...
Processing Hepatitis A...
Processing Hepatitis B...
Processing Hepatitis C...
Processing Hiatus hernia...
Processing High blood pressure (hypertension)...
Processing High cholesterol...
Processing HIV...
Processing Hives...
Processing Hodgkin lymphoma...
Processing Hodgkin lymphoma: Teenagers and young adults...
Processing Huntington’s disease...
Processing Hydrocephalus...
Processing Hyperglycaemia (high blood sugar)...
Processing Hyperhidrosis...
Processing Hypoglycaemia (low blood sugar)...
Processing Hypothermia (low body temperature)...
All data saved to all_diseases_data.json


#########################################################################################################################################################################

In [23]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual i_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('I', 'Idiopathic pulmonary fibrosis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/idiopathic-pulmonary-fibrosis/')
('I', 'If your child has cold or flu symptoms', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/if-your-child-has-cold-or-flu-symptoms/')
('I', 'Impetigo', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/impetigo/')
('I', 'Indigestion', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/indigestion/')
('I', 'Ingrown toenail', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/ingrown-toenail/')
('I', 'Infertility', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/infertility/')
('I', 'Inflammatory bowel disease (IBD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/inflammatory-bowel-disease-ibd')
('I', 'Inherited heart conditions', 'https://www.nhsinform.

In [25]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('I', 'Idiopathic pulmonary fibrosis', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/idiopathic-pulmonary-fibrosis/'),
('I', 'If your child has cold or flu symptoms', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/if-your-child-has-cold-or-flu-symptoms/'),
('I', 'Impetigo', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/impetigo/'),
('I', 'Indigestion', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/indigestion/'),
('I', 'Ingrown toenail', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/ingrown-toenail/'),
('I', 'Infertility', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/infertility/'),
('I', 'Inflammatory bowel disease (IBD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/inflammatory-bowel-disease-ibd'),
('I', 'Inherited heart conditions', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/inherited-heart-conditions/'),
('I', 'Insomnia', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/insomnia/'),
('I', 'Intoeing (pigeon toe) in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/intoeing-pigeon-toe-in-children-and-young-people/'),
('I', 'Iron deficiency anaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/iron-deficiency-anaemia/'),
('I', 'Irritable bowel syndrome (IBS)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/irritable-bowel-syndrome-ibs/'),
('I', 'Itching', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/itchy-skin/'),
('I', 'Itchy bottom', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/itchy-bottom/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_I= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_I[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_I.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_I, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Idiopathic pulmonary fibrosis...
Processing If your child has cold or flu symptoms...
Processing Impetigo...
Processing Indigestion...
Processing Ingrown toenail...
Processing Infertility...
Processing Inflammatory bowel disease (IBD)...
Processing Inherited heart conditions...
Processing Insomnia...
Processing Intoeing (pigeon toe) in children and young people...
Processing Iron deficiency anaemia...
Processing Irritable bowel syndrome (IBS)...
Processing Itching...
Processing Itchy bottom...
All data saved to all_diseases_data.json


#######################################################################################################################################################3

In [26]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual j_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('J', 'Joint hypermobility', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/joint-hypermobility/')


In [27]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('J', 'Joint hypermobility', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/joint-hypermobility/')
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_J= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_J[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_J.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_J, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Joint hypermobility...
All data saved to all_diseases_data.json


###########################################################################################################################################################################

In [28]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual k_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('K', 'Kaposi’s sarcoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/kaposis-sarcoma/')
('K', 'Kidney cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/kidney-cancer/')
('K', 'Kidney infection', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/kidney-infection/')
('K', 'Kidney stones', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/kidney-stones/')


In [29]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('K', 'Kaposi’s sarcoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/kaposis-sarcoma/'),
('K', 'Kidney cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/kidney-cancer/'),
('K', 'Kidney infection', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/kidney-infection/'),
('K', 'Kidney stones', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/kidney-stones/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_K= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_K[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_K.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_K, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Kaposi’s sarcoma...
Processing Kidney cancer...
Processing Kidney infection...
Processing Kidney stones...
All data saved to all_diseases_data.json


#########################################################################################################################################################

In [30]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual l_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('L', 'Labyrinthitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/labyrinthitis/')
('L', 'Lactose intolerance', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/lactose-intolerance/')
('L', 'Laryngeal (larynx) cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/laryngeal-larynx-cancer/')
('L', 'Laryngitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/laryngitis/')
('L', 'Late miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/late-miscarriage/')
('L', 'Leg cramps', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/leg-cramps/')
('L', 'Legionnaires’ disease', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/legionnaires-disease/')
('L', 'Lichen planus', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/lichen-pl

In [31]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('L', 'Labyrinthitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/labyrinthitis/'),
('L', 'Lactose intolerance', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/lactose-intolerance/'),
('L', 'Laryngeal (larynx) cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/laryngeal-larynx-cancer/'),
('L', 'Laryngitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/laryngitis/'),
('L', 'Late miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/late-miscarriage/'),
('L', 'Leg cramps', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/leg-cramps/'),
('L', 'Legionnaires’ disease', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/legionnaires-disease/'),
('L', 'Lichen planus', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/lichen-planus/'),
('L', 'Limb girdle muscular dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/limb-girdle-muscular-dystrophy/'),
('L', 'Lipoedema', 'https://www.nhsinform.scot/healthy-living/womens-health/middle-years-around-25-to-50-years/long-term-conditions/lipoedema/'),
('L', 'Liver cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/liver-cancer/'),
('L', 'Liver disease', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/liver-disease/'),
('L', 'Long-term effects of COVID-19', 'https://www.nhsinform.scot/long-term-effects-of-covid-19-long-covid/'),
('L', 'Loss of libido', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/loss-of-libido/'),
('L', 'Low blood pressure (hypotension)', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/low-blood-pressure-hypotension/'),
('L', 'Lung cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/lung-cancer/'),
('L', 'Lupus', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/lupus/'),
('L', 'Lyme disease', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/lyme-disease/'),
('L', 'Lymphoedema', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/lymphoedema/'),
('L', 'Lymphogranuloma venereum (LGV)', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/lymphogranuloma-venereum-lgv/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_L= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_L[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_L.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_L, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Labyrinthitis...
Processing Lactose intolerance...
Processing Laryngeal (larynx) cancer...
Processing Laryngitis...
Processing Late miscarriage...
Processing Leg cramps...
Processing Legionnaires’ disease...
Processing Lichen planus...
Processing Limb girdle muscular dystrophy...
Processing Lipoedema...
Processing Liver cancer...
Processing Liver disease...
Processing Long-term effects of COVID-19...
Processing Loss of libido...
Processing Low blood pressure (hypotension)...
Processing Lung cancer...
Processing Lupus...
Processing Lyme disease...
Processing Lymphoedema...
Processing Lymphogranuloma venereum (LGV)...
All data saved to all_diseases_data.json


#############################################################################################################################################################

In [32]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual m_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('M', 'Malaria', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/malaria/')
('M', 'Malignant brain tumour (cancerous)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/malignant-brain-tumour-cancerous/')
('M', 'Malnutrition', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/malnutrition/')
('M', 'Managing genital symptoms', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/managing-genital-symptoms/')
('M', 'Measles', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/measles/')
('M', 'Mechanical neck pain', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/mechanical-neck-pain/')
('M', 'Meningitis', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/meningitis/')
('M', 'Meniere’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/

In [33]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('M', 'Malaria', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/malaria/'),
('M', 'Malignant brain tumour (cancerous)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/malignant-brain-tumour-cancerous/'),
('M', 'Malnutrition', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/malnutrition/'),
('M', 'Managing genital symptoms', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/managing-genital-symptoms/'),
('M', 'Measles', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/measles/'),
('M', 'Mechanical neck pain', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/mechanical-neck-pain/'),
('M', 'Meningitis', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/meningitis/'),
('M', 'Meniere’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/menieres-disease/'),
('M', 'Menopause', 'https://www.nhsinform.scot/healthy-living/womens-health/later-years-around-50-years-and-over#menopause-and-post-menopause-health'),
('M', 'Mesothelioma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/mesothelioma/'),
('M', 'Metacarpal fracture of the hand', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/metacarpal-fracture-of-the-hand/'),
('M', 'Middle ear infection (otitis media)', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/middle-ear-infection-otitis-media/'),
('M', 'Migraine', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/migraine/'),
('M', 'Minor head injury', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/minor-head-injury/'),
('M', 'Miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/miscarriage/'),
('M', 'Molar pregnancy', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/molar-pregnancy/'),
('M', 'Motor neurone disease (MND)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/motor-neurone-disease-mnd/'),
('M', 'Mouth cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/mouth-cancer/'),
('M', 'Mouth ulcer', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/mouth-ulcer/'),
('M', 'Multiple myeloma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/multiple-myeloma/'),
('M', 'Multiple sclerosis (MS)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/multiple-sclerosis-ms/'),
('M', 'Multiple system atrophy (MSA)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/multiple-system-atrophy-msa/'),
('M', 'Mumps', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/mumps/'),
('M', 'Munchausen’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/munchausens-syndrome/'),
('M', 'Muscular dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/muscular-dystrophy/'),
('M', 'Myalgic encephalomyelitis (ME) or chronic fatigue syndrome (CFS)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/myalgic-encephalomyelitis-me-or-chronic-fatigue-syndrome-cfs/'),
('M', 'Myasthenia gravis', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/myasthenia-gravis/'),
('M', 'Mycoplasma genitalium (Mgen)', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/mycoplasma-genitalium-mgen/'),
('M', 'Myotonic dystrophy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/myotonic-dystrophy/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_M= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_M[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_M.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_M, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Malaria...
Processing Malignant brain tumour (cancerous)...
Processing Malnutrition...
Processing Managing genital symptoms...
Processing Measles...
Processing Mechanical neck pain...
Processing Meningitis...
Processing Meniere’s disease...
Processing Menopause...
Processing Mesothelioma...
Processing Metacarpal fracture of the hand...
Processing Middle ear infection (otitis media)...
Processing Migraine...
Processing Minor head injury...
Processing Miscarriage...
Processing Molar pregnancy...
Processing Motor neurone disease (MND)...
Processing Mouth cancer...
Processing Mouth ulcer...
Processing Multiple myeloma...
Processing Multiple sclerosis (MS)...
Processing Multiple system atrophy (MSA)...
Processing Mumps...
Processing Munchausen’s syndrome...
Processing Muscular dystrophy...
Processing Myalgic encephalomyelitis (ME) or chronic fatigue syndrome (CFS)...
Processing Myasthenia gravis...
Processing Mycoplasma genitalium (Mgen)...
Processing Myotonic dystrophy...
All da

#######################################################################################################################################

In [34]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual n_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('N', 'Nasal and sinus cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/nasal-and-sinus-cancer/')
('N', 'Nasopharyngeal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/nasopharyngeal-cancer/')
('N', 'Neck injury', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/neck-injury/')
('N', 'Neck problems', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/neck-problems/')
('N', 'Neuroendocrine tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/neuroendocrine-tumours/')
('N', 'Non-alcoholic fatty liver disease (NAFLD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/non-alcoholic-fatty-liver-disease-nafld/')
('N', 'Non-Hodgkin lymphoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/non

In [35]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('N', 'Nasal and sinus cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/nasal-and-sinus-cancer/'),
('N', 'Nasopharyngeal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/nasopharyngeal-cancer/'),
('N', 'Neck injury', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/neck-injury/'),
('N', 'Neck problems', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/neck-problems/'),
('N', 'Neuroendocrine tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/neuroendocrine-tumours/'),
('N', 'Non-alcoholic fatty liver disease (NAFLD)', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/non-alcoholic-fatty-liver-disease-nafld/'),
('N', 'Non-Hodgkin lymphoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/non-hodgkin-lymphoma/'),
('N', 'Norovirus', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/norovirus/'),
('N', 'Nosebleed', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/nosebleed/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_N= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_N[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_N.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_N, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Nasal and sinus cancer...
Processing Nasopharyngeal cancer...
Processing Neck injury...
Processing Neck problems...
Processing Neuroendocrine tumours...
Processing Non-alcoholic fatty liver disease (NAFLD)...
Processing Non-Hodgkin lymphoma...
Processing Norovirus...
Processing Nosebleed...
All data saved to all_diseases_data.json


####################################################################################################################################################

In [36]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual o_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('O', 'Obesity', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/obesity/')
('O', 'Obsessive compulsive disorder (OCD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/obsessive-compulsive-disorder-ocd/')
('O', 'Obstructive sleep apnoea', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/obstructive-sleep-apnoea/')
('O', 'Oculopharyngeal muscular dystrophy (OPMD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/oculopharyngeal-muscular-dystrophy-opmd/')
('O', 'Oesophageal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/oesophageal-cancer/')
('O', 'Oral thrush in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/oral-thrush-in-adults/')
('O', 'Osteoarthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/osteoarth

In [37]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('O', 'Obesity', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/obesity/'),
('O', 'Obsessive compulsive disorder (OCD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/obsessive-compulsive-disorder-ocd/'),
('O', 'Obstructive sleep apnoea', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/obstructive-sleep-apnoea/'),
('O', 'Oculopharyngeal muscular dystrophy (OPMD)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/muscular-dystrophy/oculopharyngeal-muscular-dystrophy-opmd/'),
('O', 'Oesophageal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/oesophageal-cancer/'),
('O', 'Oral thrush in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/oral-thrush-in-adults/'),
('O', 'Osteoarthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/osteoarthritis/'),
('O', 'Osteoarthritis of the hip', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/osteoarthritis-of-the-hip/'),
('O', 'Osteoarthritis of the knee', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/osteoarthritis-of-the-knee/'),
('O', 'Osteoarthritis of the hand', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/osteoarthritis-of-the-hand/'),
('O', 'Osteoporosis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/osteoporosis/'),
('O', 'Osteosarcoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/osteosarcoma/'),
('O', 'Outer ear infection (otitis externa)', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/otitis-externa/'),
('O', 'Ovarian cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/ovarian-cancer/'),
('O', 'Ovarian cancer: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/ovarian-cancer-teenagers-and-young-adults/'),
('O', 'Ovarian cyst', 'https://www.nhsinform.scot/healthy-living/womens-health/middle-years-around-25-to-50-years/periods-and-menstrual-health/ovarian-cyst/'),
('O', 'Overactive thyroid', 'https://www.nhsinform.scot/illnesses-and-conditions/glands/overactive-thyroid/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_O= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_O[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_O.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_O, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Obesity...
Processing Obsessive compulsive disorder (OCD)...
Processing Obstructive sleep apnoea...
Processing Oculopharyngeal muscular dystrophy (OPMD)...
Processing Oesophageal cancer...
Processing Oral thrush in adults...
Processing Osteoarthritis...
Processing Osteoarthritis of the hip...
Processing Osteoarthritis of the knee...
Processing Osteoarthritis of the hand...
Processing Osteoporosis...
Processing Osteosarcoma...
Processing Outer ear infection (otitis externa)...
Processing Ovarian cancer...
Processing Ovarian cancer: Teenagers and young adults...
Processing Ovarian cyst...
Processing Overactive thyroid...
All data saved to all_diseases_data.json


########################################################################################################################################################

In [38]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual p_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('P', 'Pain in the ball of the foot', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/pain-in-the-ball-of-the-foot/')
('P', 'Paget’s disease of the breast', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/pagets-disease-of-the-breast/')
('P', 'Pancreatic cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/pancreatic-cancer/')
('P', 'Panic disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/panic-disorder/')
('P', 'Parkinson’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/parkinsons-disease/')
('P', 'Patau’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/chromosomal-conditions/pataus-syndrome/')
('P', 'Patellofemoral pain syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/patellofemoral-pai

In [39]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('P', 'Pain in the ball of the foot', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/pain-in-the-ball-of-the-foot/'),
('P', 'Paget’s disease of the breast', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/pagets-disease-of-the-breast/'),
('P', 'Pancreatic cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/pancreatic-cancer/'),
('P', 'Panic disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/panic-disorder/'),
('P', 'Parkinson’s disease', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/parkinsons-disease/'),
('P', 'Patau’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/chromosomal-conditions/pataus-syndrome/'),
('P', 'Patellofemoral pain syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/patellofemoral-pain-syndrome/'),
('P', 'Pelvic inflammatory disease', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/pelvic-inflammatory-disease/'),
('P', 'Pelvic organ prolapse', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/pelvic-organ-prolapse/'),
('P', 'Penile cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/penile-cancer/'),
('P', 'Peripheral neuropathy', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/peripheral-neuropathy/'),
('P', 'Personality disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/personality-disorder/'),
('P', 'PIMS', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/coronavirus-covid-19/complications/pims/'),
('P', 'Plantar heel pain', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/plantar-heel-pain/'),
('P', 'Pleurisy', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/pleurisy/'),
('P', 'Pneumonia', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/pneumonia/'),
('P', 'Polio', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/polio/'),
('P', 'Polycystic ovary syndrome (PCOS)', 'https://www.nhsinform.scot/healthy-living/womens-health/girls-and-young-women-puberty-to-around-25/periods-and-menstrual-health/polycystic-ovary-syndrome-pcos/'),
('P', 'Polymyalgia rheumatica', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/polymyalgia-rheumatica/'),
('P', 'Post-concussion syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/post-concussion-syndrome-pcs/'),
('P', 'Post-polio syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/post-polio-syndrome/'),
('P', 'Popliteal cysts in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/popliteal-cysts-in-children-and-young-people/'),
('P', 'Positional talipes in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/positional-talipes-in-children-and-young-people/'),
('P', 'Post-traumatic stress disorder (PTSD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/post-traumatic-stress-disorder-ptsd/'),
('P', 'Postural orthostatic tachycardia syndrome (PoTS)', 'https://www.nhsinform.scot/long-covid/postural-orthostatic-tachycardia-syndrome-pots/'),
('P', 'Postnatal depression', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/postnatal-depression/'),
('P', 'Pregnancy and baby', 'https://www.nhsinform.scot/ready-steady-baby/'),
('P', 'Pressure ulcers', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/pressure-ulcers/'),
('P', 'Progressive supranuclear palsy (PSP)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/progressive-supranuclear-palsy-psp/'),
('P', 'Prostate cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/prostate-cancer/'),
('P', 'Psoriasis', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/psoriasis/'),
('P', 'Psoriatic arthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/psoriatic-arthritis/'),
('P', 'Psychosis', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/psychosis/'),
('P', 'Psychotic depression', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/psychotic-depression/'),
('P', 'Pubic lice', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/pubic-lice/'),
('P', 'Pulmonary hypertension', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/pulmonary-hypertension/'),
('P', 'Phobias', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/phobias/'),
('P', 'Pelvic girdle pain', 'https://www.nhsinform.scot/healthy-living/pregnancy-and-maternity/pelvic-girdle-pain/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_P= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_P[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_P.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_P, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Pain in the ball of the foot...
Processing Paget’s disease of the breast...
Processing Pancreatic cancer...
Processing Panic disorder...
Processing Parkinson’s disease...
Processing Patau’s syndrome...
Processing Patellofemoral pain syndrome...
Processing Pelvic inflammatory disease...
Processing Pelvic organ prolapse...
Processing Penile cancer...
Processing Peripheral neuropathy...
Processing Personality disorder...
Processing PIMS...
Processing Plantar heel pain...
Processing Pleurisy...
Processing Pneumonia...
Processing Polio...
Processing Polycystic ovary syndrome (PCOS)...
Processing Polymyalgia rheumatica...
Processing Post-concussion syndrome...
Processing Post-polio syndrome...
Processing Popliteal cysts in children and young people...
Processing Positional talipes in children and young people...
Processing Post-traumatic stress disorder (PTSD)...
Processing Postural orthostatic tachycardia syndrome (PoTS)...
Processing Postnatal depression...
Processing Pregnancy 

################################################################################################################################################################

In [40]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual r_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('R', 'Rare conditions', 'https://www.nhsinform.scot/illnesses-and-conditions/rare-conditions/')
('R', 'Rare tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/rare-tumours/')
('R', 'Raynaud’s phenomenon', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/raynauds-phenomenon/')
('R', 'Reactive arthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/reactive-arthritis/')
('R', 'Recovering from a cardiac arrest', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/recovering-from-a-cardiac-arrest/')
('R', 'Recurrent miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/recurrent-miscarriage/')
('R', 'Restless legs syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/restless-legs-syndrome/')
('R', 'Respiratory syncytial virus (RSV)', 'h

In [41]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('R', 'Rare conditions', 'https://www.nhsinform.scot/illnesses-and-conditions/rare-conditions/'),
('R', 'Rare tumours', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/rare-tumours/'),
('R', 'Raynaud’s phenomenon', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/raynauds-phenomenon/'),
('R', 'Reactive arthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/reactive-arthritis/'),
('R', 'Recovering from a cardiac arrest', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/recovering-from-a-cardiac-arrest/'),
('R', 'Recurrent miscarriage', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/losing-a-baby/recurrent-miscarriage/'),
('R', 'Restless legs syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/restless-legs-syndrome/'),
('R', 'Respiratory syncytial virus (RSV)', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/respiratory-syncytial-virus-rsv/'),
('R', 'Rhabdomyosarcoma', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-children/rhabdomyosarcoma/'),
('R', 'Rheumatoid arthritis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/rheumatoid-arthritis/'),
('R', 'Ringworm and other fungal infections', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/ringworm-and-other-fungal-infections/'),
('R', 'Rosacea', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/rosacea/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_R= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_R[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_R.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_R, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Rare conditions...
Processing Rare tumours...
Processing Raynaud’s phenomenon...
Processing Reactive arthritis...
Processing Recovering from a cardiac arrest...
Processing Recurrent miscarriage...
Processing Restless legs syndrome...
Processing Respiratory syncytial virus (RSV)...
Processing Rhabdomyosarcoma...
Processing Rheumatoid arthritis...
Processing Ringworm and other fungal infections...
Processing Rosacea...
All data saved to all_diseases_data.json


################################################################################################################################################################

In [42]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual s_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('S', 'Scabies', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/scabies/')
('S', 'Scarlet fever', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/scarlet-fever/')
('S', 'Schizophrenia', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/schizophrenia/')
('S', 'Sciatica', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/sciatica/')
('S', 'About scoliosis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/scoliosis/about-scoliosis/')
('S', 'Seasonal affective disorder (SAD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/seasonal-affective-disorder-sad/')
('S', 'Sepsis', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/sepsis/')
('S', 'Septic shock', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/septic-shock/')
('S', 'Severe h

In [43]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('S', 'Scabies', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/scabies/'),
('S', 'Scarlet fever', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/scarlet-fever/'),
('S', 'Schizophrenia', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/schizophrenia/'),
('S', 'Sciatica', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/sciatica/'),
('S', 'About scoliosis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/scoliosis/about-scoliosis/'),
('S', 'Seasonal affective disorder (SAD)', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/seasonal-affective-disorder-sad/'),
('S', 'Sepsis', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/sepsis/'),
('S', 'Septic shock', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/septic-shock/'),
('S', 'Severe head injury', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/head-and-neck-injuries/severe-head-injury/'),
('S', 'Shigella', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/shigella/'),
('S', 'Shingles', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/shingles/'),
('S', 'Shortness of breath', 'https://www.nhsinform.scot/illnesses-and-conditions/lungs-and-airways/shortness-of-breath/'),
('S', 'Sickle cell disease', 'https://www.nhsinform.scot/illnesses-and-conditions/blood-and-lymph/sickle-cell-disease/'),
('S', 'Sinusitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/sinusitis/'),
('S', 'Sjogren’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/sjogrens-syndrome/'),
('S', 'Skin cancer (melanoma)', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/skin-cancer-melanoma/'),
('S', 'Skin cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/skin-cancer/'),
('S', 'Skin light sensitivity (photosensitivity)', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/skin-light-sensitivity/skin-light-sensitivity-photosensitivity/'),
('S', 'Skin rashes in children', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/skin-rashes-in-children/'),
('S', 'Slapped cheek syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/slapped-cheek-syndrome/'),
('S', 'Social anxiety disorder', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/social-anxiety-disorder/'),
('S', 'Soft tissue sarcomas', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/soft-tissue-sarcomas/'),
('S', 'Soft tissue sarcomas: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/soft-tissue-sarcomas-teenagers-and-young-adults/'),
('S', 'Sore throat', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/sore-throat/'),
('S', 'Spina bifida', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/spina-bifida/'),
('S', 'Spinal stenosis', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/spinal-stenosis/'),
('S', 'Spleen problems and spleen removal', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/spleen-problems-and-spleen-removal/'),
('S', 'Stillbirth', 'https://www.nhsinform.scot/illnesses-and-conditions/pregnancy-and-childbirth/during-a-pregnancy/stillbirth/'),
('S', 'Stomach ache and abdominal pain', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/stomach-ache-and-abdominal-pain/'),
('S', 'Stomach cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/stomach-cancer/'),
('S', 'Stomach ulcer', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/stomach-ulcer/'),
('S', 'Streptococcus A (strep A)', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/streptococcus-a-strep-a/'),
('S', 'Stress, anxiety and low mood', 'https://www.nhsinform.scot/healthy-living/mental-wellbeing/'),
('S', 'Stroke', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/stroke/'),
('S', 'Subacromial pain syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/subacromial-pain-syndrome/'),
('S', 'Sudden infant death syndrome (SIDS)', 'https://www.nhsinform.scot/ready-steady-baby/early-parenthood/going-home/sleeping-safely/'),
('S', 'Suicide', 'https://www.nhsinform.scot/surviving-suicidal-thoughts/getting-help-with-your-suicidal-thoughts/'),
('S', 'Sunbed and tanning safety', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/sunbed-and-tanning-safety/'),
('S', 'Sunburn', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/skin-injuries/sunburn/'),
('S', 'Supraventricular tachycardia', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/supraventricular-tachycardia/'),
('S', 'Swollen glands', 'https://www.nhsinform.scot/illnesses-and-conditions/glands/swollen-glands/'),
('S', 'Syphilis', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/syphilis/'),
('S', 'Self-harm', 'https://www.nhsinform.scot/illnesses-and-conditions/mental-health/self-harm/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_S= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_S[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_S.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_S, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Scabies...
Processing Scarlet fever...
Processing Schizophrenia...
Processing Sciatica...
Processing About scoliosis...
Processing Seasonal affective disorder (SAD)...
Processing Sepsis...
Processing Septic shock...
Processing Severe head injury...
Processing Shigella...
Processing Shingles...
Processing Shortness of breath...
Processing Sickle cell disease...
Processing Sinusitis...
Processing Sjogren’s syndrome...
Processing Skin cancer (melanoma)...
Processing Skin cancer...
Processing Skin light sensitivity (photosensitivity)...
Processing Skin rashes in children...
Processing Slapped cheek syndrome...
Processing Social anxiety disorder...
Processing Soft tissue sarcomas...
Processing Soft tissue sarcomas: Teenagers and young adults...
Processing Sore throat...
Processing Spina bifida...
Processing Spinal stenosis...
Processing Spleen problems and spleen removal...
Processing Stillbirth...
Processing Stomach ache and abdominal pain...
Processing Stomach cancer...
Process

#############################################################################################################################################################################

In [44]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual t_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('T', 'Tennis elbow', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/tennis-elbow/')
('T', 'Testicular cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/testicular-cancer/')
('T', 'Testicular cancer: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/testicular-cancer-teenagers-and-young-adults/')
('T', 'Testicular lumps and swellings', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/testicular-lumps-and-swellings/')
('T', 'Thirst', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/thirst/')
('T', 'Threadworms', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/threadworms/')
('T', 'Thrush', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/thrush/')
('T', 'Thumb fracture', 'htt

In [1]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('T', 'Tennis elbow', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/tennis-elbow/'),
('T', 'Testicular cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/testicular-cancer/'),
('T', 'Testicular cancer: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/testicular-cancer-teenagers-and-young-adults/'),
('T', 'Testicular lumps and swellings', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/testicular-lumps-and-swellings/'),
('T', 'Thirst', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/thirst/'),
('T', 'Threadworms', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/threadworms/'),
('T', 'Thrush', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/thrush/'),
('T', 'Thumb fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/thumb-fracture/'),
('T', 'Thyroid cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/thyroid-cancer/'),
('T', 'Thyroid cancer: Teenagers and young adults', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-teenagers-and-young-adults/thyroid-cancer-teenagers-and-young-adults/'),
('T', 'Tick bites', 'https://www.nhsinform.scot/illnesses-and-conditions/injuries/skin-injuries/tick-bites/'),
('T', 'Tinnitus', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/tinnitus/'),
('T', 'Tonsillitis', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/tonsillitis/'),
('T', 'Tooth decay', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/tooth-decay/'),
('T', 'Toothache', 'https://www.nhsinform.scot/illnesses-and-conditions/mouth/toothache/'),
('T', 'Tourette’s syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/tourettes-syndrome/'),
('T', 'Transient ischaemic attack (TIA)', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/transient-ischaemic-attack-tia/'),
('T', 'Transverse myelitis', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/transverse-myelitis/'),
('T', 'Trichomonas infection', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/trichomonas-infection/'),
('T', 'Trigeminal neuralgia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/trigeminal-neuralgia/'),
('T', 'Trigger thumb or trigger finger in children and young people', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/children-and-young-peoples-muscles-bones-and-joints/trigger-thumb-in-children-and-young-people/'),
('T', 'Tuberculosis (TB)', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/tuberculosis-tb/'),
('T', 'Type 1 diabetes', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/type-1-diabetes/'),
('T', 'Type 2 diabetes', 'https://www.nhsinform.scot/illnesses-and-conditions/diabetes/type-2-diabetes/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_T= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_T[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_T.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_T, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Tennis elbow...
Processing Testicular cancer...
Processing Testicular cancer: Teenagers and young adults...
Processing Testicular lumps and swellings...
Processing Thirst...
Processing Threadworms...
Processing Thrush...
Processing Thumb fracture...
Processing Thyroid cancer...
Processing Thyroid cancer: Teenagers and young adults...
Processing Tick bites...
Processing Tinnitus...
Processing Tonsillitis...
Processing Tooth decay...
Processing Toothache...
Processing Tourette’s syndrome...
Processing Transient ischaemic attack (TIA)...
Processing Transverse myelitis...
Processing Trichomonas infection...
Processing Trigeminal neuralgia...
Processing Trigger thumb or trigger finger in children and young people...
Processing Tuberculosis (TB)...
Processing Type 1 diabetes...
Processing Type 2 diabetes...
All data saved to all_diseases_data.json


#############################################################################################################################################################3

In [7]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual u_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('U', 'Ulcerative colitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/inflammatory-bowel-disease-ibd/ulcerative-colitis/')
('U', 'Underactive thyroid', 'https://www.nhsinform.scot/illnesses-and-conditions/glands/underactive-thyroid/')
('U', 'Urinary incontinence', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-incontinence/')
('U', 'Urinary incontinence in women', 'https://www.nhsinform.scot/healthy-living/womens-health/middle-years-around-25-to-50-years/pelvic-health/urinary-incontinence-in-women/')
('U', 'Urinary tract infection (UTI)', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-tract-infection-uti/')
('U', 'Urinary tract infection (UTI) in children', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-tract-infection-uti-in-children/')
('U', 'Urticaria (hives)', 'https://www.nhsinform.scot/illnesses-and-co

In [8]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('U', 'Ulcerative colitis', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/inflammatory-bowel-disease-ibd/ulcerative-colitis/'),
('U', 'Underactive thyroid', 'https://www.nhsinform.scot/illnesses-and-conditions/glands/underactive-thyroid/'),
('U', 'Urinary incontinence', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-incontinence/'),
('U', 'Urinary incontinence in women', 'https://www.nhsinform.scot/healthy-living/womens-health/middle-years-around-25-to-50-years/pelvic-health/urinary-incontinence-in-women/'),
('U', 'Urinary tract infection (UTI)', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-tract-infection-uti/'),
('U', 'Urinary tract infection (UTI) in children', 'https://www.nhsinform.scot/illnesses-and-conditions/kidneys-bladder-and-prostate/urinary-tract-infection-uti-in-children/'),
('U', 'Urticaria (hives)', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/hives/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_U= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_U[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_U.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_U, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Ulcerative colitis...
Processing Underactive thyroid...
Processing Urinary incontinence...
Processing Urinary incontinence in women...
Processing Urinary tract infection (UTI)...
Processing Urinary tract infection (UTI) in children...
Processing Urticaria (hives)...
All data saved to all_diseases_data.json


##############################################################################################################################################

In [9]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual v_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('V', 'Vaginal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/vaginal-cancer/')
('V', 'Vaginal discharge', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/vaginal-discharge/')
('V', 'Varicose eczema', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/varicose-eczema/')
('V', 'Varicose veins', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/varicose-veins/')
('V', 'Vascular dementia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia/types-of-dementia/vascular-dementia/')
('V', 'Venous leg ulcer', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/venous-leg-ulcer/')
('V', 'Vertigo', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/vertigo/')
('V', 'Vitamin B12 or folate deficiency anaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/vitamin-b12-or-folate-deficiency-anaem

In [10]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('V', 'Vaginal cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/vaginal-cancer/'),
('V', 'Vaginal discharge', 'https://www.nhsinform.scot/illnesses-and-conditions/sexual-and-reproductive/vaginal-discharge/'),
('V', 'Varicose eczema', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/varicose-eczema/'),
('V', 'Varicose veins', 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/varicose-veins/'),
('V', 'Vascular dementia', 'https://www.nhsinform.scot/illnesses-and-conditions/brain-nerves-and-spinal-cord/dementia/types-of-dementia/vascular-dementia/'),
('V', 'Venous leg ulcer', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/venous-leg-ulcer/'),
('V', 'Vertigo', 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/vertigo/'),
('V', 'Vitamin B12 or folate deficiency anaemia', 'https://www.nhsinform.scot/illnesses-and-conditions/nutritional/vitamin-b12-or-folate-deficiency-anaemia/'),
('V', 'Vomiting in adults', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/vomiting-in-adults/'),
('V', 'Vomiting in children and babies', 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/vomiting-in-children-and-babies/'),
('V', 'Vulval cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/vulval-cancer/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_V= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_V[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_V.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_V, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Vaginal cancer...
Processing Vaginal discharge...
Processing Varicose eczema...
Processing Varicose veins...
Processing Vascular dementia...
Processing Venous leg ulcer...
Processing Vertigo...
Processing Vitamin B12 or folate deficiency anaemia...
Processing Vomiting in adults...
Processing Vomiting in children and babies...
Processing Vulval cancer...
All data saved to all_diseases_data.json


############################################################################################################################################################################

In [11]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual w_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('W', 'Warts and verrucas', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/warts-and-verrucas/')
('W', 'Whiplash', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/whiplash/')
('W', 'Whooping cough', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/whooping-cough/')
('W', 'Wolff-Parkinson-White syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/wolff-parkinson-white-syndrome/')
('W', 'Womb (uterus) cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/womb-uterus-cancer/')
('W', 'Wrist fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/wrist-fracture/')


In [13]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('W', 'Warts and verrucas', 'https://www.nhsinform.scot/illnesses-and-conditions/skin-hair-and-nails/warts-and-verrucas/'),
('W', 'Whiplash', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/neck-and-back-problems-and-conditions/whiplash/'),
('W', 'Whooping cough', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/whooping-cough/'),
('W', 'Wolff-Parkinson-White syndrome', 'https://www.nhsinform.scot/illnesses-and-conditions/cardiovascular-disease/heart-disease/wolff-parkinson-white-syndrome/'),
('W', 'Womb (uterus) cancer', 'https://www.nhsinform.scot/illnesses-and-conditions/cancer/cancer-types-in-adults/womb-uterus-cancer/'),
('W', 'Wrist fracture', 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/arm-shoulder-and-hand-problems-and-conditions/wrist-fracture/'),
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_W= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_W[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_W.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_W, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Warts and verrucas...
Processing Whiplash...
Processing Whooping cough...
Processing Wolff-Parkinson-White syndrome...
Processing Womb (uterus) cancer...
Processing Wrist fracture...
All data saved to all_diseases_data.json


###############################################################################################################################################################

In [14]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual y_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('Y', 'Yellow fever', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/yellow-fever/')


In [15]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('Y', 'Yellow fever', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/yellow-fever/')
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_Y= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_Y[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_Y.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_Y, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Yellow fever...
All data saved to all_diseases_data.json


#############################################################################################################################################################3

In [16]:
def parse_page(soup):
    rows = []

    class_list = soup.find_all("div", {"class": "az_list_indivisual z_detail"})
    
    for div in class_list:
        letter = div.find("h2").text.strip() if div.find("h2") else ""
        ul = div.find("ul")
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                if a:
                    name = a.text.strip()
                    link = a.get("href")
                    rows.append((letter, name, link))
    return rows

disease_data = parse_page(soup)
for item in disease_data[:]:
    print(item)


('Z', 'Zika virus', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/zika-virus/')


In [17]:
import requests
from bs4 import BeautifulSoup
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.243 Safari/537.36"
}

def get_sections_key_value(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    h2_list = soup.find_all("h2", class_="wp-block-heading")

    data = {}
    for idx, h2 in enumerate(h2_list):
        title = h2.get_text(strip=True)
        next_h2 = h2_list[idx + 1] if idx + 1 < len(h2_list) else None

        paragraphs = []
        for sibling in h2.find_next_siblings():
            if sibling == next_h2:
                break
            if sibling.name == "p":
                text = sibling.get_text(strip=True)
                if text and text != ".":
                    paragraphs.append(text)
        
        if paragraphs:
            data[title] = paragraphs

    return data

# قائمة الروابط (مثال مختصر)
disease_links = [
('Z', 'Zika virus', 'https://www.nhsinform.scot/illnesses-and-conditions/infections-and-poisoning/zika-virus/')
]
# إنشاء JSON واحد لكل الأمراض
all_diseases_data_Z= {}

for letter, name, link in disease_links:
    print(f"Processing {name}...")
    sections = get_sections_key_value(link)
    all_diseases_data_Z[name] = sections

# حفظ النتيجة في ملف JSON
with open("all_diseases_data_Z.json", "w", encoding="utf-8") as f:
    json.dump(all_diseases_data_Z, f, ensure_ascii=False, indent=4)

print("All data saved to all_diseases_data.json")


Processing Zika virus...
All data saved to all_diseases_data.json


###########################################################################################################################################################

In [11]:
import json
import glob

# مسار الملفات، هنا كل ملفات JSON في نفس المجلد
files = glob.glob("*.json")

merged_data = []

for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        # لو البيانات قائمة (Array)، نضيف كل العناصر
        if isinstance(data, list):
            merged_data.extend(data)
        # لو البيانات كائن (Object)، نحولها لقائمة من العناصر
        elif isinstance(data, dict):
            merged_data.append(data)

# حفظ البيانات المدمجة في ملف واحد
with open("merged.json", "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=4)

print("تم الدمج! تم إنشاء merged.json")


تم الدمج! تم إنشاء merged.json


In [12]:
import json
import glob

files = glob.glob("*.json")
merged_data = []

for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        if isinstance(data, list):
            merged_data.extend(data)
        elif isinstance(data, dict):
            for key, value in data.items():
                merged_data.append({key: value})

# عدد الأمراض
print("عدد الأمراض الكلي:", len(merged_data))

# حفظ البيانات المدمجة
with open("merged.json", "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=4)


عدد الأمراض الكلي: 468


In [5]:
import json
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
collection = db["diseases"]


with open("merged.json", "r", encoding="utf-8") as f:#قراءة json  
    data = json.load(f)

collection.insert_many(data)# insert data to mongo 


print("تم التحويل لـ MongoDB بنجاح!")



تم التحويل لـ MongoDB بنجاح!


In [1]:
import json
import os

# المجلد اللي فيه ملفات JSON
folder_path = "DATA"

# لو عايز تحسب لكل ملف على حدة
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # لو عندك المفتاح 'disease'
        diseases = [entry['disease'] for entry in data if 'disease' in entry]
        unique_diseases = set(diseases)
        
        print(f"File: {filename} --> Total diseases: {len(unique_diseases)}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'DATA'

############################################################################################################################################